In [ ]:
pip list

In [1]:
# -*- encoding:utf-8 -*-
#-*- coding: cp950 -*-

import os
import sys

import pyspark
import random

import matplotlib.pyplot as plt
import numpy as np
import numpy
from matplotlib import cm
import pandas as pd
from numpy import concatenate
from pandas import concat

from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import col  
import pyspark.sql.types
from pyspark.sql.types import DoubleType
from pyspark.sql import Row
from pyspark.sql.types import Row
from pyspark.sql.functions import udf

from pyspark import since, SparkContext, keyword_only

from pyspark.ml.common import _java2py, _py2java
from pyspark.ml.wrapper import _jvm
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaEstimator, JavaModel
from pyspark.ml.param.shared import *
from pyspark.ml.common import inherit_doc

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import QuantileDiscretizer, VectorSlicer
from pyspark.ml.feature import ChiSqSelector

from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.feature import Binarizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.stat import Summarizer

from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import PCA
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.feature import DCT
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import OneHotEncoder, Normalizer, StandardScaler
from pyspark.ml.feature import MinMaxScaler, MaxAbsScaler, Bucketizer
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.feature import SQLTransformer

from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan

from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import LDA
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import PowerIterationClustering


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LogisticRegression , OneVsRest
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import FMClassifier

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import FMRegressor

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.fpm import FPGrowth
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD



In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)


21/07/26 07:29:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
dt= spark.read.format('csv').option("header",'true').load("/home/jovyan/work/pySpark/pySpark_james/data/free-zipcode-database-Primary.csv")

print('data:',dt.count()) 
dt.show(5)

data: 42522
+-------+-----------+--------+-----+------------+-----+------+-----------------+-------------+---------------+-------------------+----------+
|Zipcode|ZipCodeType|    City|State|LocationType|  Lat|  Long|         Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|
+-------+-----------+--------+-----+------------+-----+------+-----------------+-------------+---------------+-------------------+----------+
|  00705|   STANDARD|AIBONITO|   PR|     PRIMARY|18.14|-66.26|NA-US-PR-AIBONITO|        false|           null|               null|      null|
|  00610|   STANDARD|  ANASCO|   PR|     PRIMARY|18.28|-67.14|  NA-US-PR-ANASCO|        false|           null|               null|      null|
|  00611|     PO BOX| ANGELES|   PR|     PRIMARY|18.28|-66.79| NA-US-PR-ANGELES|        false|           null|               null|      null|
|  00612|   STANDARD| ARECIBO|   PR|     PRIMARY|18.45|-66.73| NA-US-PR-ARECIBO|        false|           null|               null|      

In [4]:
dt.printSchema()

root
 |-- Zipcode: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: string (nullable = true)
 |-- TaxReturnsFiled: string (nullable = true)
 |-- EstimatedPopulation: string (nullable = true)
 |-- TotalWages: string (nullable = true)



In [5]:
def replace_question(x):
    return ("0" if x=="?" else x)  
replace_question= udf(replace_question)

dt.select("AIBONITO").show

# df=dt.select(['url','urlid','alchemy_category']+
#              [replace_question(col(column)).cast("double").alias(column)  
#                 for column in dt.columns[3:] ] )

# categoryIndexer = StringIndexer(inputCol='alchemy_category', outputCol="alchemy_category_Index")
# categoryTransformer=categoryIndexer.fit(df)

# df1=categoryTransformer.transform(df)
# assemblerInputs =['alchemy_category_Index'] + dt.columns[4:-1]
# assembler = VectorAssembler( inputCols=assemblerInputs,outputCol="features")
# df3=assembler.transform(df1)

# traindf, testdf = df.randomSplit([0.7, 0.3])
# traindf.cache()
# testdf.cache()

# mod=GBTClassifier()
# PL= Pipeline(stages=[categoryIndexer,assembler,mod])
# PLmod= PL.fit(traindf)
# pred=PLmod.transform(testdf)
# evaluator = BinaryClassificationEvaluator(
#                               rawPredictionCol="rawPrediction",
#                               labelCol="label",  
#                               metricName="areaUnderROC")

# pred = PLmod.transform(testdf)
# DescDict = { 0: " (ephemeral)",
#            1: " (evergreen)"}
# for data in pred.select('url','prediction').take(5):
#     print( "URL:  " +str(data[0])+"\n" + "==>predict:"+ str(data[1])+ " note:"+DescDict[data[1]] +"\n")

# auc= evaluator.evaluate(pred)
# print('AUC=',auc)

AnalysisException: cannot resolve '`AIBONITO`' given input columns: [City, Decommisioned, EstimatedPopulation, Lat, Location, LocationType, Long, State, TaxReturnsFiled, TotalWages, ZipCodeType, Zipcode];
'Project ['AIBONITO]
+- Relation[Zipcode#16,ZipCodeType#17,City#18,State#19,LocationType#20,Lat#21,Long#22,Location#23,Decommisioned#24,TaxReturnsFiled#25,EstimatedPopulation#26,TotalWages#27] csv
